In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

# Define a simple model
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.layer1 = nn.Linear(10000, 10000)
        self.layer2 = nn.Linear(10000, 1)

    def forward(self, x):
        x = torch.relu(self.layer1(x))
        x = self.layer2(x)
        return x

model = MyModel().cuda()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Dummy data
inputs = torch.randn(64, 10000).to("cuda")
targets = torch.randn(64, 1).to("cuda")

# Training loop
model.train()
for epoch in tqdm(range(1000)):
    optimizer.zero_grad()
    outputs = model(inputs)
    loss = nn.MSELoss()(outputs, targets)
    loss.backward()
    optimizer.step()

100%|██████████| 1000/1000 [00:07<00:00, 132.66it/s]


In [1]:
import deepspeed
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

# Define a simple model
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.layer1 = nn.Linear(10000, 10000)
        self.layer2 = nn.Linear(10000, 1)

    def forward(self, x):
        x = torch.relu(self.layer1(x))
        x = self.layer2(x)
        return x

model = MyModel()
# Configure DeepSpeed
config = {
    "train_batch_size": 64,
    "gradient_accumulation_steps": 1,
    "fp16": {
        "enabled": False
    },
   "optimizer": {
        "type": "Adam",
        "params": {
            "lr": 0.001
        }
    }
}

optimizer = optim.Adam(model.parameters(), lr=0.001)

# Initialize DeepSpeed
model_engine, optimizer, _, _ = deepspeed.initialize(model=model,
                                                      model_parameters=model.parameters(),
                                                      config=config,
                                                     optimizer=optimizer,
                                                     args=None
                                                     )
# Dummy data
inputs = torch.randn(64, 10000).to("cuda")
targets = torch.randn(64, 1).to("cuda")
# Training loop with DeepSpeed
model_engine.train()
for epoch in tqdm(range(1000)):
    optimizer.zero_grad()
    outputs = model_engine(inputs)
    loss = nn.MSELoss()(outputs, targets)
    model_engine.backward(loss)
    model_engine.step()

[2024-04-15 18:45:58,364] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2024-04-15 18:46:03,511] [INFO] [logging.py:96:log_dist] [Rank -1] DeepSpeed info: version=0.14.0, git-hash=unknown, git-branch=unknown
[2024-04-15 18:46:03,512] [INFO] [comm.py:637:init_distributed] cdb=None
[2024-04-15 18:46:03,513] [INFO] [comm.py:652:init_distributed] Not using the DeepSpeed or dist launchers, attempting to detect MPI environment...
[2024-04-15 18:46:05,364] [INFO] [comm.py:702:mpi_discovery] Discovered MPI settings of world_rank=0, local_rank=0, world_size=1, master_addr=10.151.0.1, master_port=29500
[2024-04-15 18:46:05,365] [INFO] [comm.py:668:init_distributed] Initializing TorchBackend in DeepSpeed with backend nccl
[2024-04-15 18:46:08,403] [INFO] [logging.py:96:log_dist] [Rank 0] DeepSpeed Flops Profiler Enabled: False
[2024-04-15 18:46:08,404] [INFO] [logging.py:96:log_dist] [Rank 0] Using client Optimizer as basic optimizer
[2024-04-15 18

  1%|          | 8/1000 [00:07<10:52,  1.52it/s]  

[2024-04-15 18:46:15,938] [INFO] [logging.py:96:log_dist] [Rank 0] step=10, skipped=0, lr=[0.001], mom=[(0.9, 0.999)]
[2024-04-15 18:46:15,952] [INFO] [timer.py:260:stop] epoch=0/micro_step=10/global_step=10, RunningAvgSamplesPerSec=4401.916253289932, CurrSamplesPerSec=4419.418109976951, MemAllocated=1.14GB, MaxMemAllocated=1.88GB


  2%|▏         | 15/1000 [00:07<04:50,  3.39it/s]

[2024-04-15 18:46:16,085] [INFO] [logging.py:96:log_dist] [Rank 0] step=20, skipped=0, lr=[0.001], mom=[(0.9, 0.999)]
[2024-04-15 18:46:16,098] [INFO] [timer.py:260:stop] epoch=0/micro_step=20/global_step=20, RunningAvgSamplesPerSec=4426.507355925279, CurrSamplesPerSec=4434.1645908355085, MemAllocated=1.14GB, MaxMemAllocated=1.88GB


...

 99%|█████████▉| 988/1000 [00:21<00:00, 67.39it/s]

[2024-04-15 18:46:30,433] [INFO] [logging.py:96:log_dist] [Rank 0] step=990, skipped=0, lr=[0.001], mom=[(0.9, 0.999)]
[2024-04-15 18:46:30,446] [INFO] [timer.py:260:stop] epoch=0/micro_step=990/global_step=990, RunningAvgSamplesPerSec=4463.924788990304, CurrSamplesPerSec=4416.291660497179, MemAllocated=1.14GB, MaxMemAllocated=1.88GB


100%|█████████▉| 995/1000 [00:21<00:00, 67.19it/s]

[2024-04-15 18:46:30,580] [INFO] [logging.py:96:log_dist] [Rank 0] step=1000, skipped=0, lr=[0.001], mom=[(0.9, 0.999)]
[2024-04-15 18:46:30,594] [INFO] [timer.py:260:stop] epoch=0/micro_step=1000/global_step=1000, RunningAvgSamplesPerSec=4463.89466765887, CurrSamplesPerSec=4463.06414391647, MemAllocated=1.14GB, MaxMemAllocated=1.88GB


100%|██████████| 1000/1000 [00:21<00:00, 45.73it/s]


In [5]:
import deepspeed
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

# Define a simple model
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.layer1 = nn.Linear(10000, 10000)
        self.layer2 = nn.Linear(10000, 1)

    def forward(self, x):
        x = torch.relu(self.layer1(x))
        x = self.layer2(x)
        return x

def training_function():
    model = MyModel()
    # Configure DeepSpeed
    config = {
        "train_batch_size": 64,
        "gradient_accumulation_steps": 1,
        "fp16": {
            "enabled": False
            },
        "optimizer": {
        "type": "Adam",
        "params": {
            "lr": 0.001
            }
        }
    }

    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Initialize DeepSpeed
    model_engine, optimizer, _, _ = deepspeed.initialize(model=model,
                                                      model_parameters=model.parameters(),
                                                      config=config,
                                                     optimizer=optimizer,
                                                     args=None
                                                     )
    # Dummy data
    inputs = torch.randn(64, 10000).to("cuda")
    targets = torch.randn(64, 1).to("cuda")
    # Training loop with DeepSpeed
    model_engine.train()
    for epoch in tqdm(range(1000)):
        optimizer.zero_grad()
        outputs = model_engine(inputs)
        loss = nn.MSELoss()(outputs, targets)
        model_engine.backward(loss)
        model_engine.step()

from accelerate import notebook_launcher
notebook_launcher(training_function, num_processes=1)

Launching training on one GPU.
[2024-04-15 18:52:32,980] [INFO] [logging.py:96:log_dist] [Rank 0] DeepSpeed info: version=0.14.0, git-hash=unknown, git-branch=unknown
[2024-04-15 18:52:33,068] [INFO] [logging.py:96:log_dist] [Rank 0] DeepSpeed Flops Profiler Enabled: False
[2024-04-15 18:52:33,069] [INFO] [logging.py:96:log_dist] [Rank 0] Using client Optimizer as basic optimizer
[2024-04-15 18:52:33,070] [INFO] [logging.py:96:log_dist] [Rank 0] Removing param_group that has no 'params' in the basic Optimizer
[2024-04-15 18:52:33,070] [INFO] [logging.py:96:log_dist] [Rank 0] DeepSpeed Basic Optimizer = Adam
[2024-04-15 18:52:33,070] [INFO] [logging.py:96:log_dist] [Rank 0] DeepSpeed Final Optimizer = Adam
[2024-04-15 18:52:33,071] [INFO] [logging.py:96:log_dist] [Rank 0] DeepSpeed using client LR scheduler
[2024-04-15 18:52:33,071] [INFO] [logging.py:96:log_dist] [Rank 0] DeepSpeed LR Scheduler = None
[2024-04-15 18:52:33,071] [INFO] [logging.py:96:log_dist] [Rank 0] step=0, skipped=0,

  1%|          | 7/1000 [00:00<00:14, 66.54it/s]

[2024-04-15 18:52:33,237] [INFO] [logging.py:96:log_dist] [Rank 0] step=10, skipped=0, lr=[0.001], mom=[(0.9, 0.999)]
[2024-04-15 18:52:33,250] [INFO] [timer.py:260:stop] epoch=0/micro_step=10/global_step=10, RunningAvgSamplesPerSec=4356.560346174211, CurrSamplesPerSec=4324.651705305215, MemAllocated=2.26GB, MaxMemAllocated=3.01GB


  1%|▏         | 14/1000 [00:00<00:14, 65.99it/s]

[2024-04-15 18:52:33,386] [INFO] [logging.py:96:log_dist] [Rank 0] step=20, skipped=0, lr=[0.001], mom=[(0.9, 0.999)]
[2024-04-15 18:52:33,399] [INFO] [timer.py:260:stop] epoch=0/micro_step=20/global_step=20, RunningAvgSamplesPerSec=4387.817412735631, CurrSamplesPerSec=4422.330411861614, MemAllocated=2.26GB, MaxMemAllocated=3.01GB


...

 99%|█████████▊| 987/1000 [00:14<00:00, 68.35it/s]

[2024-04-15 18:52:47,706] [INFO] [logging.py:96:log_dist] [Rank 0] step=990, skipped=0, lr=[0.001], mom=[(0.9, 0.999)]
[2024-04-15 18:52:47,718] [INFO] [timer.py:260:stop] epoch=0/micro_step=990/global_step=990, RunningAvgSamplesPerSec=4462.446414867401, CurrSamplesPerSec=4419.127090741473, MemAllocated=2.26GB, MaxMemAllocated=3.01GB


 99%|█████████▉| 994/1000 [00:14<00:00, 67.84it/s]

[2024-04-15 18:52:47,855] [INFO] [logging.py:96:log_dist] [Rank 0] step=1000, skipped=0, lr=[0.001], mom=[(0.9, 0.999)]
[2024-04-15 18:52:47,867] [INFO] [timer.py:260:stop] epoch=0/micro_step=1000/global_step=1000, RunningAvgSamplesPerSec=4462.318661138759, CurrSamplesPerSec=4424.152550473836, MemAllocated=2.26GB, MaxMemAllocated=3.01GB


100%|██████████| 1000/1000 [00:14<00:00, 67.71it/s]


In [6]:
!torchrun --nproc_per_node 2 deepspeed.py

[2024-04-15 18:54:11,894] torch.distributed.run: [WARNING] 
[2024-04-15 18:54:11,894] torch.distributed.run: [WARNING] *****************************************
[2024-04-15 18:54:11,894] torch.distributed.run: [WARNING] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
[2024-04-15 18:54:11,894] torch.distributed.run: [WARNING] *****************************************
[W socket.cpp:436] [c10d] The server socket has failed to bind to [::]:29500 (errno: 98 - Address already in use).
[W socket.cpp:436] [c10d] The server socket has failed to bind to 0.0.0.0:29500 (errno: 98 - Address already in use).
[E socket.cpp:472] [c10d] The server socket has failed to listen on any local network address.
Traceback (most recent call last):
  File "/scratch/qualis/miniconda3/envs/dp/bin/torchrun", line 33, in <module>
    sys.exit(load_entry_poi